In [1]:
import numpy as np
import parsl
from parsl import python_app
from parsl.config import Config
from parsl.executors.threads import ThreadPoolExecutor
import pandas as pd
import os
import re
import pandas as pd
from transformers import BertModel, BertTokenizer
import torch
from sklearn.metrics.pairwise import cosine_similarity
import time 
import threading
import hdbscan
import psutil
import GPUtil
from time import sleep
from bertopic import BERTopic
from datetime import datetime

/home/lyncoln/Git/similaridade_workflow/venv_linux/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def monitor_resources(interval, path_name):
    while not stop_thread.is_set():
        # Obter o timestamp atual
        current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

        # Monitorando CPU e memória
        cpu_usage = psutil.cpu_percent(interval=interval)
        memory_info = psutil.virtual_memory()
        memory_usage = memory_info.used / (1024 ** 2)  # Convertendo de bytes para MB

        # Monitorando GPU
        gpus = GPUtil.getGPUs()
        gpu_info = [(gpu.load, gpu.memoryUsed) for gpu in gpus]
        gpu_load = gpu_info[0][0]
        gpu_memory_used = gpu_info[0][1]

        new_row = pd.DataFrame([{
            'Timestamp': current_time,
            'CPU': cpu_usage,
            'Memory': memory_usage,
            'GPU': gpu_load,
            'GPU_Memory': gpu_memory_used
        }])

        if not os.path.isfile(path_name):
            df = pd.DataFrame(columns=["Timestamp", "CPU", "Memory", "GPU", "GPU_Memory"])
            df.to_csv(path_name, index=False)
        else:
            df = pd.read_csv(path_name)

        df = pd.concat([df, new_row], ignore_index=True)
        df.to_csv(path_name, index=False)

        # Limpando a memória
        del df

        sleep(interval)

In [3]:
torch.cuda.set_per_process_memory_fraction(0.99)
config = Config(executors=[ThreadPoolExecutor()])
parsl.load(config)

In [4]:
def extractName(text):
  return re.findall(r'"name"\s*:\s*"([^"]+)"',text)

def extractTag(text):
  matches = re.findall(r'\"tags\": \[(.*?)\]', text, re.DOTALL)
  matches = re.findall(r'\"(.*?)\"', matches[0].strip(','))
  return matches

def extractDescription(text):
  return re.findall(r'"description"\s*:\s*"([^"]+)"',text)

def extractLabel(text):
    return re.findall(r'"label"\s*:\s*"([^"]+)"', text)


In [5]:
os.chdir(os.path.join(os.pardir,os.pardir, 'workflows_galaxy'))
nomes_arquivos = []
diretorio = os.getcwd()
print(diretorio)
for item in os.listdir(diretorio):
    caminho_completo = os.path.join(diretorio, item)
    if os.path.isfile(caminho_completo):
        nomes_arquivos.append(item)

print(len(nomes_arquivos))

/home/lyncoln/Git/similaridade_workflow/workflows_galaxy
1014


In [6]:
dic_workflows = {}
@python_app
def processar_arquivo(arquivo):
    names = []
    tags = []
    descriptions = []
    labels = []

    # Carregar o arquivo JSON
    with open(arquivo, 'r') as file:
        fileName = file.name
        dados = file.read()

        # Extração das informações
        names.extend(extractName(dados))
        tags.extend(extractTag(dados))
        descriptions.extend(extractDescription(dados))
        labels.extend(extractLabel(dados))
        all_results = names + tags + descriptions + labels


    return all_results

# Inicia o processamento dos arquivos em paralelo
futures = [processar_arquivo(arquivo) for arquivo in nomes_arquivos]

# Obter os resultados
resultados = [future.result() for future in futures]

for arquivo, all_results in zip(nomes_arquivos, resultados):
  dic_workflows[arquivo] = all_results

In [7]:
df = pd.DataFrame(list(dic_workflows.items()), columns=['name', 'text'])
df

,name,text
0,0c86c39dcd9e08c6.json,[Project_CP]
1,692a2b0bb818336d.json,[Workflow constructed from history 'K22063917 ...
2,3680984663c813e1.json,"[dhfr, Bowtie2, output, bowtie2, MPileup, outp..."
3,49f8b32c3206f76c.json,[Workflow for Genomic Data Science with Galaxy...
4,25adc55d2a26e34b.json,[GigaScience Example 1B -- aye-aye FST (import...
...,...,...
1009,d5cba8a5ba6880fd.json,"[BRACA2 - primer design, exon, Input dataset, ..."
1010,e0da87cadb1e6d5f.json,"[2 peaks-result, data file 2 and more peaks 16..."
1011,fbf75fbb72b488bd.json,"['BBL735_Lab2(Olympic)_AT', olympics.tsv, Inpu..."
1012,0d8eb75d28fa7df2.json,"[Desanka Lazic_Project, Coriell-NA12880_R2.fas..."


In [7]:
os.chdir("..")
os.chdir("scr")
os.chdir("similaridade")

execution_times = []

In [8]:

df = pd.DataFrame(list(dic_workflows.items()), columns=['name', 'text'])

# Medir o tempo de execução
start_time = time.time()

stop_thread = threading.Event()
monitor_thread = threading.Thread(target=monitor_resources, args=(1.0,f"hardware_top3_topico_descricao.csv",))    
monitor_thread.daemon = True
monitor_thread.start()

# Carregar o modelo e o tokenizer

df['topic_bert'] = ""

tokenizer = BertTokenizer.from_pretrained('google-bert/bert-large-uncased')
model = BertModel.from_pretrained('google-bert/bert-large-uncased')

for i in range(len(df['text'])):
    print(i)
    # Contar o número de palavras no texto atual
    list_text = df['text'][i]
    n_words = sum(len(words.split()) for words in list_text)

    if n_words > 10:
        # Instanciar o BERTopic
        topic_model = BERTopic(embedding_model=model, hdbscan_model=hdbscan.HDBSCAN(min_cluster_size=2))
        topics_total, probabilities = topic_model.fit_transform(df['text'][i])

        if len(topic_model.get_topics()) > 2:
            topic_model.reduce_topics(df['text'][i], nr_topics=2)

        # Obter as 10 palavras principais do primeiro tópico
        topics = topic_model.get_topics()
        topic = [word for word, _ in topics[0][:10]]
        df['topic_bert'][i] = topic

    else: #Se tem menos de 10 palavras, então o texto é o próprio tópico
        df['topic_bert'][i] = df['text'][i]

    topic_model = ''

# Definir o dispositivo (GPU ou CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

def text_to_embedding(text):
    encoded_input = tokenizer(text, return_tensors='pt', max_length=512, padding=True, truncation=True)
    with torch.no_grad():
        model_output = model(**encoded_input)
    # Pegar a média dos embeddings de todos os tokens para representar o texto
    return model_output.last_hidden_state.mean(dim=1).squeeze().numpy()

df['topic_bert_str'] = df['topic_bert'].apply(lambda x: ' '.join(x))
embeddings = np.vstack(df['topic_bert_str'].apply(text_to_embedding))

# Calcular a similaridade de cosseno entre todos os vetores
cosine_sim = cosine_similarity(embeddings)

embeddings_array = np.vstack(embeddings)

# Calcular a matriz de similaridade
similarity_matrix = cosine_similarity(embeddings_array)

# Identificar os índices dos textos mais similares para cada texto
similar_indices = similarity_matrix.argsort(axis=1)[:, :-top_x-2:-1]  # Selecionar os top_x mais similares excluindo o próprio texto

# Remover o índice do próprio texto
corrected_similar_indices = []
corrected_similar_tags = []
for idx, indices in enumerate(similar_indices):
    filtered_indices = [index for index in indices if index != idx][:3]  # Exclui o próprio e pega os top_x mais similares
    filtered_tags = [df.iloc[index]['Tags'] for index in filtered_indices]  # Obter as tags dos textos mais similares
    corrected_similar_indices.append(filtered_indices)
    corrected_similar_tags.append(filtered_tags)

# Criar coluna no DataFrame para os índices dos textos mais similares
df[f'top3_descricao'] = corrected_similar_indices

# Criar coluna no DataFrame para as tags dos textos mais similares
df[f'top3_tags_descricao'] = corrected_similar_tags

# Calcular a média das similaridades dos textos mais similares para cada texto
mean_similarities = []
for idx, indices in enumerate(corrected_similar_indices):
    similarities = [similarity_matrix[idx, i] for i in indices]
    mean_similarity = np.mean(similarities)
    mean_similarities.append(mean_similarity)

df[f'mean_similarity_top3_descricao'] = mean_similarities

# Calcular o tempo de execução
execution_time = time.time() - start_time

# Adicionar o tempo de execução ao DataFrame de tempos de execução
execution_times.append({'top_x': top_x, 'execution_time_descricao': execution_time})

torch.cuda.empty_cache()
stop_thread.set()
monitor_thread.join()

df.to_csv(f"results_topicos_bert.csv")

0
1


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

2
3


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


4


...

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


170


/home/lyncoln/Git/similaridade_workflow/venv_linux/lib/python3.8/site-packages/scipy/sparse/linalg/_eigen/arpack/arpack.py:1600: RuntimeWarning: k >= N for N * N square matrix. Attempting to use scipy.linalg.eigh instead.
  warnings.warn("k >= N for N * N square matrix. "
/home/lyncoln/Git/similaridade_workflow/venv_linux/lib/python3.8/site-packages/scipy/sparse/linalg/_eigen/arpack/arpack.py:1600: RuntimeWarning: k >= N for N * N square matrix. Attempting to use scipy.linalg.eigh instead.
  warnings.warn("k >= N for N * N square matrix. "
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


TypeError: Cannot use scipy.linalg.eigh for sparse A with k >= N. Use scipy.linalg.eigh(A.toarray()) or reduce k.

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [10]:
df

,name,text,topic_bert
0,0c86c39dcd9e08c6.json,[Project_CP],[Project_CP]
1,692a2b0bb818336d.json,[Workflow constructed from history 'K22063917 ...,"[out_file1, filter, fastqc, vcffilter, output,..."
2,3680984663c813e1.json,"[dhfr, Bowtie2, output, bowtie2, MPileup, outp...","[dhfr, Bowtie2, output, bowtie2, MPileup, outp..."
3,49f8b32c3206f76c.json,[Workflow for Genomic Data Science with Galaxy...,"[fastqc, fastq, text_file, html_file, r2, bwa,..."
4,25adc55d2a26e34b.json,[GigaScience Example 1B -- aye-aye FST (import...,"[genome_diversity, output, aye, snps, histogra..."
...,...,...,...
1009,d5cba8a5ba6880fd.json,"[BRACA2 - primer design, exon, Input dataset, ...",
1010,e0da87cadb1e6d5f.json,"[2 peaks-result, data file 2 and more peaks 16...",
1011,fbf75fbb72b488bd.json,"['BBL735_Lab2(Olympic)_AT', olympics.tsv, Inpu...",
1012,0d8eb75d28fa7df2.json,"[Desanka Lazic_Project, Coriell-NA12880_R2.fas...",


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 